In [ ]:
import pandas as pd
import numpy as np

# Parámetros
capital = 10_000_000
tasa_interes = 0.04
v = 1 / (1 + tasa_interes)
edades_objetivo = [20, 30, 40, 50, 60, 70]

# Cargar archivo con proyecciones diferenciadas por sexo
archivo = "LeeCarter Tabla con Proyecciones y Esperanzas.xlsx"  # ajustar nombre si es distinto
tabla_hombres = pd.read_excel(archivo, sheet_name='Hombres', index_col=0)
tabla_mujeres = pd.read_excel(archivo, sheet_name='Mujeres', index_col=0)

# Asegurar que los índices sean edades como enteros
tabla_hombres.index = tabla_hombres.index.astype(int)
tabla_mujeres.index = tabla_mujeres.index.astype(int)

# Función para cálculo de rentas vitalicias
def calcular_rentas(tabla, edades, capital, v):
    resultados = {"Edad": edades}
    for anio in tabla.columns:
        primas = []
        for edad in edades:
            if edad not in tabla.index:
                primas.append(np.nan)
                continue
            qx = tabla.loc[edad:, anio].dropna().values
            if len(qx) < 2:
                primas.append(np.nan)
                continue
            lx = [1.0]
            for q in qx[:-1]:
                lx.append(lx[-1] * (1 - q))
            lx = np.array(lx)
            Lx = (lx[:-1] + lx[1:]) / 2
            v_pot = v ** np.arange(len(Lx))
            a_x = np.sum(Lx * v_pot)
            prima = round(a_x * capital, 2)
            primas.append(prima)
        resultados[int(anio)] = primas
    return pd.DataFrame(resultados)

# Calcular resultados para hombres y mujeres
df_rentas_hombres = calcular_rentas(tabla_hombres, edades_objetivo, capital, v)
df_rentas_mujeres = calcular_rentas(tabla_mujeres, edades_objetivo, capital, v)
